# 05. Firebase와 연동
- 본 문서에서는 Firebase와 연동하는 방법, 그리고 연동 후 BigQuery에 데이터가 어떻게 쌓이는지에 대해 알려드리겠습니다
- Firebase Blaze(종량제) 요금제를 사용해야 BigQuery와 연동할 수 있습니다!

## 1) Firebase Console로 이동
- [Firebase Console](https://console.firebase.google.com/)
<img src="../images/010_connect_firebase_01.png" width="700" height="700">

- '프로젝트 추가'를 클릭해서 Firebase 프로젝트를 만들어주세요 :)
- 그 후, test라는 모양같이 생성된 프로젝트를 클릭해주세요!

<img src="../images/010_connect_firebase_02.png" width="700" height="700">

## 2) 앱에 Firebase 추가
- 저는 Android 앱에 Firebase 추가를 해보겠습니다! 클릭!

<img src="../images/010_connect_firebase_03.png" width="500" height="700">

- 패키지 이름과 닉네임을 설정해줍니다!

<img src="../images/010_connect_firebase_04.png" width="500" height="700">

- 루트 디렉토리에 google-services.json 파일을 이동시켜주세요!


<img src="../images/010_connect_firebase_05.png" width="500" height="700">

- build.gradle을 수정해 플러그인을 사용해주세요!
- 이렇게 하면 Android와 Firebase와 연동은 끝입니다!!!!

<img src="../images/010_connect_firebase_06.png" width="700" height="700">

- 이렇게 앱이 설정되었습니다!
- 이벤트 구성이 궁금하신 분들은 [안드로이드](https://firebase.google.com/docs/analytics/android/events), [iOS](https://firebase.google.com/docs/analytics/ios/events)를 참고해주세요!

## 3) Firebase와 BigQuery 연동
- 네모친 톱니바퀴 버튼을 클릭해주세요. 그 후, 계정 연결을 눌러주세요!

<img src="../images/010_connect_firebase_07.png" width="900" height="700">

- 프로젝트 및 링크 업그레이드를 클릭해주세요
- 그 후 요금제를 변경해야한다는 창이 뜰거에요!! Firebase Blaze 요금제를 사용해야 연결할 수 있습니다

<img src="../images/010_connect_firebase_08.png" width="500" height="700">

- Blaze 선택!

<img src="../images/010_connect_firebase_09.png" width="400" height="700">

- BigQuery 내보내기에 광고 식별자 포함을 한 후 계속을 눌러주면 연동이 끝납니다!

## 4) BigQuery에서 데이터 탐색
- 연결 설정을 한 이후 바로 Dataset, Table이 생성되는 것은 아니고 데이터가 어느정도 쌓인 후 생성됩니다
- 당일의 데이터는 Dataset 이름.app_events_intraday_YYYYMMDD 형태로 테이블이 생성된 후, 당일이 지나면 intraday가 사라진 이름으로 복제됩니다
- Data가 모두 들어오는 시간이 2일-3일정도 소요됩니다(1일만에 모두 들어오는 경우도 있습니다. 앱의 유저들에 따라 다른 것 같습니다)
- Firebase와 연동된 BigQuery의 스키마는 아래와 같이 표현됩니다

<img src="../images/010_connect_firebase_10.png" width="500" height="700">

<img src="../images/010_connect_firebase_11.png" width="500" height="700">

<img src="../images/010_connect_firebase_12.png" width="500" height="700">

- 앱의 국가정보, device 정보까지 알아서 알려줍니다! ( 따로 설정하지 않아도 )
- 각 스키마의 대한 안내는 [스키마 안내](https://support.google.com/firebase/answer/7029846)에 자세히 나와있습니다!
- 간단히 설명드리면 하나의 테이블 안에 user_dim part와 event_dim part 2개로 구성되어 있습니다
- user_dim의 user_property는 저희가 구성하는 것으로, 유저들의 속성을 기록할 수 있습니다
- event_dim 또한 이벤트의 이름, 이벤트의 세부 파라미터를 설정할 수 있습니다
---
- 여기서 RECODE란 타입은 앞서 설명드렸듯 "하나 이상의 서브 필드를 가지는 데이터 형태"입니다( RECODE 타입의 이벤트를 2개 추출하려면 UNNEST 함수로 풀어줘야합니다)

## 5) Query 예시
- 기본적인 쿼리는 정말 간단히 작성할 수 있습니다(여기선 Legacy를 사용하겠습니다)
~~~
#legacySQL
SELECT event_dim.name as event, EXACT_COUNT_DISTINCT(user_dim.app_info.app_instance_id) as user
FROM [com_seongyun_app.app_events_20171001]
GROUP BY event
ORDER BY user DESC
~~~
- 이 쿼리는 10월 1일 각 이벤트를 사용한 유저수를 구하는 쿼리입니다. 
- COUNT할 경우 user_dim.app_info.app_instance_id를 사용했는데, Firebase에서 설정해주는 고유한 인스턴스 ID값입니다( 저희 앱 레트리카는 로그인을 안해도 사용할 수 있기에 user_id가 없는 경우가 존재해 instance_id를 사용했습니다)

~~~
#legacySQL
SELECT event_dim.name as event, EXACT_COUNT_DISTINCT(user_dim.app_info.app_instance_id) as user
FROM TABLE_DATE_RANGE(com_seongyun_app.app_events_, TIMESTAMP('20170901'), TIMESTAMP('20171001'))
GROUP BY event
ORDER BY user DESC
~~~
- 이 쿼리는 9월 1일부터 10월 1일까지 이벤트를 사용한 유저수를 구하는 쿼리입니다
- legacy SQL은 Table 범위를 TABLE_DATE_RANGE 함수를 사용해 할 수 있습니다. standardSQL은 아래와 같이 하시면 됩니다
~~~
#standardSQL
SELECT event_dim.name as event, COUNT(DISTINCT user_dim.app_info.app_instance_id) as user
FROM `com_seongyun_app.app_events_*`
WHERE _table_suffix between '20170901' and '20171001'
GROUP BY event
ORDER BY user DESC
~~~

### 간단한 쿼리의 경우 오류가 생기지 않아 쉽습니다! 그러나 event_dim의 parameter를 추출할 경우는 어떻게 해야할까요?

<img src="../images/010_connect_firebase_13.png" width="700" height="700">

- event 이름이 Save_Name이고 세부 파라미터에 name이 존재합니다 ( 앞에 고유 식별자인 instance_id 가 존재하고 각 이벤트의 유저는 다르다고 가정하겠습니다! )
- 저희는 아래와 같은 형태로 보고싶다면 어떻게 쿼리해야할까요?

| instance_id | event_name | name | 
| :---: | :---: | :---: | 
| A | Save_Name | firebase |
| B | Save_Name|  bigquery |
| C | Save_Name | firebase |
| D | Save_Name | seongyun |


~~~
#legacySQL
SELECT user_dim.app_info.app_instance_id AS instance_id, event_dim.name AS event_name, 
MAX(IF(event_dim.params.key = 'name', event_dim.params.value.string_value, null)) as name
FROM [com_seongyun_app.app_events_20171001]
GROUP BY instance_id, event_name
~~~
- 이렇게 MAX(IF ~ ) GROUP BY 를 사용해 각 파라미터의 value값을 끌어올릴 수 있습니다!